Install Required Packages

In [ ]:
!pip install tf-keras==2.19 --quiet
!pip install tensorflow==2.19 --quiet
!pip install keras==3.5 --quiet

Download and Extract Dataset

In [ ]:
import os

# Install gdown, a lightweight tool that allows direct downloads from Google Drive.
!pip install gdown

# Download the dataset ZIP file from Google Drive using its file ID.
# The -O specifies the output filename to save locally.
!gdown --id 14BIX16B7tnTgQtanvQRViG_rRVcB0HJB -O rps_dataset.zip

# Unzip the downloaded dataset quitely
# The contents are extracted into a folder named 'rps_dataset'.
!unzip -q rps_dataset.zip -d rps_dataset

# Verify that the dataset folder now exists in the Colab environment.
print("Unzipped folder exists:", os.path.exists("rps_dataset"))

# List the contents of the extracted dataset folder.
print("Contents:", os.listdir("rps_dataset") if os.path.exists("rps_dataset") else "Folder not found")

Directory Paths


In [ ]:
# TRAINING DIRECTORIES
train_rock_dir = "rps_dataset/rps/rps/rock"
train_paper_dir = "rps_dataset/rps/rps/paper"
train_scissors_dir = "rps_dataset/rps/rps/scissors"

# VALIDATION DIRECTORIES
validation_rock_dir = "rps_dataset/rps-test-set/rps-test-set/rock"
validation_paper_dir = "rps_dataset/rps-test-set/rps-test-set/paper"
validation_scissors_dir = "rps_dataset/rps-test-set/rps-test-set/scissors"

# TEST DIRECTORIES
testing_dir = "rps_dataset/rps-validation"

Convolutional Neural Network Model

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    #input shape - desired size of the image 150x150 with 3 bytes color
    #This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # incase it is 0, it will apply 3 to ensure something is returned
    tf.keras.layers.Dense(3, activation='softmax')
])

Display the Model Architecture

In [ ]:
# Display a detailed summary of the model architecture.
model.summary()

Define Custom Metrics and Compile the Model

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K

# Define metrics

# ---------------------------------------------------------
# Custom metric: Recall
# ---------------------------------------------------------

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# ---------------------------------------------------------
# Custom metric: Precision
# ---------------------------------------------------------

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

# ---------------------------------------------------------
# Compile the model
# ---------------------------------------------------------

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy',precision_m, recall_m])


Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)
testing_datagen = ImageDataGenerator(rescale=1/255)

# TRAINING DATA
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'rps_dataset/rps/rps/',
        target_size=(150, 150), #resize image 150 x 150
        batch_size=128,
        class_mode='categorical') # Since you use categorical_crossentropy loss, you need categorical labels

# VALIDATION DATA
# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        'rps_dataset/rps-test-set/rps-test-set/' ,
        target_size=(150, 150), #resize image 150 x 150
        batch_size=32,
        class_mode='categorical') # Since you use categorical_crossentropy loss, you need categorical labels

# TESTING DATA
# Flow testing images in batches of 128 using validation_datagen generator
testing_generator = testing_datagen.flow_from_directory(
        'rps_dataset/rps-validation/',
        target_size=(150, 150), #resize image 150 x 150
        batch_size=30,
        class_mode='categorical') # Since you use categorical_crossentropy loss, you need categorical labels


Custom Callback for Early Stopping

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):

    # Stops the training when the validation loss falls below 0.4
    # Check the loss
    if(logs.get('val_loss') < 0.4):

      # Stop if threshold is met
      print("\nVal Loss is lower than 0.4 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

Train the model

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8,
      callbacks=[callbacks])

Test Whether AI classifies Image into Rock, Paper or Scissors

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

# Step 1: Upload an image
uploaded = files.upload()

for fn in uploaded.keys():
    # Step 2: Load and display the image
    img_path = fn
    img = image.load_img(img_path, target_size=(150, 150))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

    # Step 3: Preprocess
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Step 4: Predict
    prediction = model.predict(img_array)

    # Step 5: Map prediction to class name
    class_labels = {v: k for k, v in train_generator.class_indices.items()}
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction)

    print(f"Prediction: {predicted_class} (confidence: {confidence:.2f})")

